In [1]:
import re
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/wilsonwid/github-repos/news-summaries/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = tfds.load("cnn_dailymail")
display(data)


Dl Completed...: 100%|██████████| 5/5 [00:19<00:00, 29.16 url/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


Dl Completed...: 100%|██████████| 5/5 [00:22<00:00, 29.16 url/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


Dl Completed...: 100%|██████████| 5/5 [00:26<00:00, 29.16 url/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, s

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()